# Application : Meteo en ligne via l’API de OpenWeatherMap
OpenWeatherMap est un service en ligne qui fournit des données météorologiques, y compris des données météorologiques actuelles, des prévisions et des données historiques aux développeurs de services Web et d'applications mobiles.


In [2]:
#Image(url= "weather.png")
!pip install pandas 
!pip install termcolor

  Using cached pandas-1.4.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
     |████████████████████████████████| 16.8 MB 15.5 MB/s            




Dans cet atelier, nous exploitions cette api afin d’extraire les données météorologique sous forme JSON et on le stocke dans un fichier CSV respectant un format bien défini.
#### Pour cela nous suivons les étapes suivantes
Depuis 2015, utiliser openweathermap nécessite de s'enregistrer. On s'authentifie ensuite en utilisant une clé. Chaque requête doit sonc être complétée par : &appid=XXXXX où XXX est la valeur de la clé à utiliser. Cette clé est disponible sur Updago.
Nous allons utiliser le service OpenWeatherMap pour récolter des prévisions météo. Ces prévisions pourront être resservies par un site Web, par exemple.
Le site http://openweathermap.org propose une API pour récolter les informations qu'il diffuse.
Prendre connaissance des possibilités de l'API
La page de documentation de l'API est accessible ici : http://openweathermap.org/api
#### Voici quelques exemples :
http://api.openweathermap.org/data/2.5/weather?q=Tunis,Tunisia&appid=XXX
Remplacez XXX par la clé API. Dans la suite, pensez à ajouter le champ appid=XXX.
Remplacer ?q=Tunis,Tunisia par la villes et le pays recherchés.
La commande weather employée ici indique que nous souhaitons obtenir les conditions météo actuelles.
Le résultat est par défaut fourni au format json. Dans le cas de la commande weather, voici les informations qui sont obtenues : Weather Data
Il y a bien sûr d'autres commandes disponibles, qu'on trouvera dans la documentation. Notez en particulier la commande forecast qui permet d'obtenir des prévisions.
## Outils Python
#### Accès à l'API
Comme indiqué dans le cours, nous allons utiliser Python pour interroger openweathermap. Le module requests sera utilisé à la place d'urllib 


#### Chargement des modules importantes


In [3]:
#from IPython.display import Image
from IPython.core.display import HTML
import os
import csv
from termcolor import colored, cprint
import datetime
import urllib.request
import pandas as pd
import re, json, requests
#os.chdir("C:\\users\\msellami\\PythonTraining\\")

#### Configuration de API et genration de url d'accès à l'API
Définition d'un fonction qui prend en parametres id de la ville, la ville, le pays pour generer un url respectant l'appel de l'<b>API OpenWeatherMAP</b> et aussi configurer l'APPID recuperé après enregistrement sur le site.
En effet, l'accès à l'api est effectué soit en utilisant ID de la ville (Tunis) récuperé via la site ( https://openweathermap.org/city/2464470) ou via un fichier json conteant la liste des pays et leur villes accessible via http://bulk.openweathermap.org/sample/city.list.json.gz.
#### Accès direct avec ID Ville/Pays
*- http://api.openweathermap.org/data/2.5/weather?id=2464470&mode=json&units=metric&APPID
#### Accès avec recherche de Ville et Pays
*-  http://api.openweathermap.org/data/2.5/weather?q=Tunis,Tunisia&mode=json&units=metric&APPID

Il faut specifier aussi les données sous forme JSON ou XML et l'unité de temperature (°C,F).
Pour Fahrenheit, on utilise unité=imperial, pour Celsius, on utilise unité= metric, et  par defaut Kelvin. 

In [4]:
def url_builder(city_id,city_name,country):
    user_api = '0e7a872c16502901b3548b5b13bf0250'  # Obtain yours form: http://openweathermap.org/
    unit = 'metric'  # For Fahrenheit use imperial, for Celsius use metric, and the default is Kelvin.
    if(city_name!=""):
        api = 'http://api.openweathermap.org/data/2.5/weather?q=' # "http://api.openweathermap.org/data/2.5/weather?q=Tunis,fr
        full_api_url = api + str(city_name) +','+ str(country)+ '&mode=json&units=' + unit + '&APPID=' + user_api
    else:
        api = 'http://api.openweathermap.org/data/2.5/weather?id='     # Search for your city ID here: http://bulk.openweathermap.org/sample/city.list.json.gz
        full_api_url = api + str(city_id) + '&mode=json&units=' + unit + '&APPID=' + user_api
   
    return full_api_url

Maintenant on passe à definir une fonction qui permet de récuperer le fichier JSON a partir de cette URL en utilisant <b>urllib.request.urlopen()</b>, <b>str.read.decode('utf-8')</b> pour l'encodage et <b>json.load() </b>pour charger une structire <b>SJON</b> a partir des fichier

In [5]:
def data_fetch(full_api_url):
    url = urllib.request.urlopen(full_api_url)
    output = url.read().decode('utf-8')
    raw_api_dict = json.loads(output)
    url.close()
    return raw_api_dict


### Convertion et formatage de heure et date
Definir une fonction qui permet de convertir un timestamp en Heure de la forme HH:MM AM/PM

In [6]:
def time_converter(time):
    converted_time = datetime.datetime.fromtimestamp(int(time)).strftime('%I:%M %p')
    return converted_time


### Extraction des champs à partir des fichiers JSON 
Mainetant, nous avons besoins de creer une fonction <b>data_organizer</b> qui prend une structure json complexe et créee une dictionnaire de données
contenant les attributs suivants:
<b>
    
*-city : La ville
    
*-country: le pays

*-temp: Temperature actuelle

*-temp_max: Temperature temp_max

*-temp_min :Temperature Min 

*-humidity ; Humidité

*-pressure ; Pression

*-sky       : Etat de Ciel 

*-sunrise  : Lever du soleil  
 
 
*-sunset : Coucher du soleil 

*-wind : Vistesse de Vent

*-wind_deg

*-dt : Date

*-cloudiness : Nuageux

</b>


In [7]:
def data_organizer(raw_api_dict):
    data = dict(
        city=raw_api_dict.get('name'),
        country=raw_api_dict.get('sys').get('country'),
        temp=raw_api_dict.get('main').get('temp'),
        temp_max=raw_api_dict.get('main').get('temp_max'),
        temp_min=raw_api_dict.get('main').get('temp_min'),
        humidity=raw_api_dict.get('main').get('humidity'),
        pressure=raw_api_dict.get('main').get('pressure'),
        sky=raw_api_dict['weather'][0]['main'],
        sunrise=time_converter(raw_api_dict.get('sys').get('sunrise')),
        sunset=time_converter(raw_api_dict.get('sys').get('sunset')),
        wind=raw_api_dict.get('wind').get('speed'),
        wind_deg=raw_api_dict.get('deg'),
        dt=time_converter(raw_api_dict.get('dt')),
        cloudiness=raw_api_dict.get('clouds').get('all')
    )
    #print (data)
    return data

# Recuperer des coordonnées des villes à partir de json avec pandas
  Maintenant on a besoin d'une fonction qui permet recuperer les coordonnées des villesd'un fichier JSON 
  

In [8]:
def WriteCSV(data):
    with open('weatherOpenMap.csv', 'a') as f:  # Just use 'w' mode in 3.x
        w = csv.DictWriter(f, data.keys())
        #w.writeheader()
        w.writerow(data)

In [12]:
villes = pd.read_json("https://raw.githubusercontent.com/data7h/scraping_cassandra_project/master/city.list.json")
villes_france = villes[villes['country'] == 'FR' ]
villes_france = villes_france.reset_index() 
villes_france = villes_france[['id','name','country','coord']]
villes_france.tail(5)

,id,name,country,coord
19960,3024670,Clenleu,FR,"{'lon': 1.87892, 'lat': 50.524231}"
19961,6449590,Autreville-sur-la-Renne,FR,"{'lon': 4.97917, 'lat': 48.117222}"
19962,2978219,Saint-Martin-sur-la-Renne,FR,"{'lon': 4.95387, 'lat': 48.145119}"
19963,3231216,Kerbonne,FR,"{'lon': -4.5012, 'lat': 48.383282}"
19964,3231196,Kéroriou,FR,"{'lon': -4.4833300000000005, 'lat': 48.383331}"


In [13]:
ids_france       = list(villes_france ['id'])
countries_france = list(villes_france ['country'])

In [14]:
if __name__ == '__main__':
    try:
        for i in range(100):
            #start = datetime.now()
            import time 
            #if i in [j for j in range(2,10,2)]:  #duration = datetime.now()- start
            #    print(i, "=> delay of 1 seconds")
            #    time.sleep(1)
            #country   = 'France'
            #city_name =  countries_france[i]
            city_id    =  ids_france[i]
            #Generation de l url
            print(colored('Generation de l url ', 'red',attrs=['bold']))
            url=url_builder(city_id,'','')
            #Invocation du API afin de recuperer les données
            print(colored('Invocation du API afin de recuperer les données', 'red',attrs=['bold']))
            data=data_fetch(url)

            data_orgnized=data_organizer(data)
            #Enregistrement des données à dans un fichier CSV 
            WriteCSV(data_orgnized)
        print("Les temperatures de toutes les villes sont scrappees :D")
    except IOError:
        print('no internet')
        print("Seullement " + str(i) + " villes ont pu étre scrapper")

df = pd.read_csv("weatherOpenMap.csv",
                     usecols    = [i for i in range(0,11)], 
                     names      = ['city','country','Temperature','temp_max','temp_min','humidity','pressure','sky','sunrise','sunset','wind_speed'],    
                     index_col  = None)
df.to_csv("weatherOpenMap.csv",index = False) 
df      

Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du API afin de recuperer les données
Generation de l url 
Invocation du

,city,country,Temperature,temp_max,temp_min,humidity,pressure,sky,sunrise,sunset,wind_speed
0,Forville,FR,-0.13,3.43,-5.70,68,1034,Clear,07:03 AM,04:26 PM,0.89
1,La Portanière,FR,0.93,6.14,-1.12,82,1028,Clear,07:00 AM,04:33 PM,1.03
2,Wittenheim,FR,0.02,2.95,-1.99,91,1036,Clouds,07:09 AM,04:14 PM,1.34
3,Wattrelos,FR,4.68,5.38,3.88,87,1036,Clouds,07:36 AM,04:20 PM,2.06
4,Vitré,FR,4.28,4.75,4.16,94,1036,Clear,07:44 AM,04:47 PM,2.74
...,...,...,...,...,...,...,...,...,...,...,...
95,Saint-Trosse,FR,1.66,2.80,-1.96,92,1032,Clear,07:32 AM,04:59 PM,1.98
96,Arrondissement de Périgueux,FR,0.46,2.31,-1.17,76,1030,Clear,07:28 AM,04:50 PM,3.39
97,Arrondissement de La Tour-du-Pin,FR,-0.12,1.39,-3.38,97,1033,Clouds,07:09 AM,04:29 PM,0.61
98,Four,FR,1.24,1.98,-1.64,87,1033,Clouds,07:10 AM,04:30 PM,2.57


In [42]:
!pip install cassandra-driver
#!pip install pandas==0.20.0.

In [20]:
import warnings

In [21]:
df = pd.read_csv("weatherOpenMap.csv")
df

,city,country,Temperature,temp_max,temp_min,humidity,pressure,sky,sunrise,sunset,wind_speed
0,Forville,FR,-0.13,3.43,-5.70,68,1034,Clear,07:03 AM,04:26 PM,0.89
1,La Portanière,FR,0.93,6.14,-1.12,82,1028,Clear,07:00 AM,04:33 PM,1.03
2,Wittenheim,FR,0.02,2.95,-1.99,91,1036,Clouds,07:09 AM,04:14 PM,1.34
3,Wattrelos,FR,4.68,5.38,3.88,87,1036,Clouds,07:36 AM,04:20 PM,2.06
4,Vitré,FR,4.28,4.75,4.16,94,1036,Clear,07:44 AM,04:47 PM,2.74
...,...,...,...,...,...,...,...,...,...,...,...
95,Saint-Trosse,FR,1.66,2.80,-1.96,92,1032,Clear,07:32 AM,04:59 PM,1.98
96,Arrondissement de Périgueux,FR,0.46,2.31,-1.17,76,1030,Clear,07:28 AM,04:50 PM,3.39
97,Arrondissement de La Tour-du-Pin,FR,-0.12,1.39,-3.38,97,1033,Clouds,07:09 AM,04:29 PM,0.61
98,Four,FR,1.24,1.98,-1.64,87,1033,Clouds,07:10 AM,04:30 PM,2.57


In [22]:
df = df [['city','Temperature','temp_max' ,'temp_min']]
#df=df.head(21)
df

,city,Temperature,temp_max,temp_min
0,Forville,-0.13,3.43,-5.70
1,La Portanière,0.93,6.14,-1.12
2,Wittenheim,0.02,2.95,-1.99
3,Wattrelos,4.68,5.38,3.88
4,Vitré,4.28,4.75,4.16
...,...,...,...,...
95,Saint-Trosse,1.66,2.80,-1.96
96,Arrondissement de Périgueux,0.46,2.31,-1.17
97,Arrondissement de La Tour-du-Pin,-0.12,1.39,-3.38
98,Four,1.24,1.98,-1.64


In [33]:
#!pip install pandas-profiling

In [34]:
#!pip install --upgrade pip
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [35]:
#from pandas_profiling import ProfileReport
#profile = ProfileReport(df, title="Résumé Descriptif", explorative=True)
#profile

In [36]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Temperature,100.0,2.9278,2.105305,-1.90,1.2075,4.05,4.6425,5.32
temp_max,100.0,4.2599,1.614730,0.19,3.0000,4.83,5.2825,9.25
temp_min,100.0,1.4088,2.698162,-5.70,-1.2575,3.09,3.4950,5.14


In [3]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
CASSANDRA_HOST = ['192.168.64.4'] 
CASSANDRA_PORT = 9042
CASSANDRA_DB = "weather"
CASSANDRA_TABLE = "countries_france"
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
def pandas_factory(colnames,rows):
    return pd.DataFrame(rows,columns=colnames)
#on force ici a repecter le datframe de pandas lors de la recuperation des données
try:
    cluster = Cluster(protocol_version=3,
                      contact_points=CASSANDRA_HOST,
                      load_balancing_policy=None,
                      port=CASSANDRA_PORT,
                      auth_provider=auth_provider)
    session = cluster.connect()
except ValueError:
    print("Oops!  échec de connexion cluster.  Try again...")

NoHostAvailable: ('Unable to connect to any servers', {'192.168.64.4:9042': OSError(None, "Tried connecting to [('192.168.64.4', 9042)]. Last error: timed out")})

In [ ]:
#creation du key space
session.execute("CREATE KEYSPACE IF NOT EXISTS weather_frApp WITH REPLICATION={'class':'SimpleStrategy','replication_factor':3};")

In [ ]:
session.execute("create table weather_frApp.weather_fr(city TEXT primary key, weather map<text,float>);")
#session.execute("insert into weather_frApp.weather_fr(city,weather) values ('Paris',{'temp_max': '90','temp_min': '90'});")

In [ ]:
rows=session.execute('Select * from weather_frApp.weather_fr;')
df_results = rows._current_rows
df_results

In [ ]:
df_city = df.drop_duplicates(subset = ['city'])[['city']]
df_city['city'] = df_city['city'].str.replace("'",' ')
df_city = df_city.head(10)
df

In [133]:
query_insert="INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ({},{});"
#session.execute("insert into weather_frApp.weather_fr(city,weather) values ('Paris',{'temp_max': '90','temp_min': '90'});")

In [134]:
for ct in df_city.index:
        row_dict = {}
        row_dict["Temperature"] = df['Temperature'][ct]
        row_dict["temp_max"]    = df['temp_max'][ct]
        row_dict["temp_min"]    = df['temp_min'][ct]
        CQL_query = query_insert.format("'"+df_city['city'][ct]+"'",row_dict)
        print(CQL_query)
        session.execute(CQL_query)

INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('Forville',{'Temperature': 0.78, 'temp_max': 3.42, 'temp_min': -4.07});
INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('La Portanière',{'Temperature': 3.41, 'temp_max': 8.79, 'temp_min': -0.12});
INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('Wittenheim',{'Temperature': -1.8, 'temp_max': 0.33, 'temp_min': -2.7});
INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('Wattrelos',{'Temperature': -0.06, 'temp_max': 3.72, 'temp_min': -1.9});
INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('Vitré',{'Temperature': 2.08, 'temp_max': 2.16, 'temp_min': 1.38});
INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('Viry-Châtillon',{'Temperature': -0.42, 'temp_max': 0.56, 'temp_min': -1.24});
INSERT INTO weather_frApp.weather_fr (city,weather) VALUES ('Villeneuve-le-Roi',{'Temperature': -0.48, 'temp_max': 0.53, 'temp_min': -1.27});
INSERT INTO weather_frApp.weather_fr (city,weather) VALU

In [135]:
#Executer une requete pour tester
rows = session.execute('SELECT * FROM weather_frApp.weather_fr; ')
df_cities = rows._current_rows
df_cities

[Row(city='Wattrelos', weather=OrderedMapSerializedKey([('Temperature', -0.05999999865889549), ('temp_max', 3.7200000286102295), ('temp_min', -1.899999976158142)])),
 Row(city='Forville', weather=OrderedMapSerializedKey([('Temperature', 0.7799999713897705), ('temp_max', 3.4200000762939453), ('temp_min', -4.070000171661377)])),
 Row(city='Viry-Châtillon', weather=OrderedMapSerializedKey([('Temperature', -0.41999998688697815), ('temp_max', 0.5600000023841858), ('temp_min', -1.2400000095367432)])),
 Row(city='Vierzon', weather=OrderedMapSerializedKey([('Temperature', -2.109999895095825), ('temp_max', -1.0299999713897705), ('temp_min', -3.6500000953674316)])),
 Row(city='La Portanière', weather=OrderedMapSerializedKey([('Temperature', 3.4100000858306885), ('temp_max', 8.789999961853027), ('temp_min', -0.11999999731779099)])),
 Row(city='Wittenheim', weather=OrderedMapSerializedKey([('Temperature', -1.7999999523162842), ('temp_max', 0.33000001311302185), ('temp_min', -2.700000047683716)])),